# Cargando las librerias requeridas

In [ ]:
import numpy as np
import pandas as pd
from plotnine import *

In [ ]:
from sklearn import datasets
wine = datasets.load_wine() #Cargando el dataset de vino
print(wine.keys()) #Analizando las variables que tiene

#Escalando los datos
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() #Creamos un objeto de la clase StandardScaler
scaled_features = scaler.fit_transform(wine.data) #Transformamos los datos

# AgglomerativeClustering

sklearn tiene varias deficiencias para este modelo. Siendo las más importantes, el definir un número de cluster y no generar un dendrograma en forma sencilla.<br>

Utilizaremos las funciones linkage, dendrogram y fcluster de la biblioteca scipy.cluster.hierarchy.<br>

La función linkage aprende el modelo definido y retorna un dendrograma<br>
linkage(y, method='single')<br>
Parámetros:
* y: son los datos del modelo
* method: ‘ward’, ‘complete’, ‘average’, ‘single’.<br><br>

Para graficar basta con llamar a la función dendrogram con el modelo aprendido<br>dendrogram(modeloAprendido,p=30,truncate_mode=None)<br>
donde truncate_mode puede ser None (muestra todo el dendrograma) o 'lastp' (corta el dendrograma después de p ramas).

Para extraer los clusters se utiliza la función fcluster<br>
fcluster(Z, t, criterion)<br>
Parámetros:
* Z: el modelo generado por la función linkage
* t: el valor donde se quiere cortar el dendrograma
* criterion=“distance”. existen otros criterios de corte, pero distance es el que nos permite generar los clusters, basados en la altura que definimos.

In [ ]:
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt

#Entrenando el modelo
modelo = shc.linkage(scaled_features, method='complete')

plt.figure(figsize=(10, 7)) #Seteando el tamaño de la figura
objeto = shc.dendrogram(modelo) #Generando el dendrograma
#objeto = shc.dendrogram(modelo,p=30,truncate_mode="lastp") #Generando el dendrograma

In [ ]:
#Extrayendo los clusters sugeridos por el método
clusters=shc.fcluster(modelo,t=8.0,criterion="distance")
clusters

In [ ]:
#Creando el objeto y aplicando PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(scaled_features)
tempData = pca.transform(scaled_features)
tempData = pd.DataFrame(tempData,columns=["PC1","PC2"])
tempData["labels"]=clusters
ggplot(tempData)+aes(x="PC1",y="PC2",color="factor(labels)")+geom_point(show_legend=False)+theme_bw()

# Trabajando con variables categoricas
En esta sección volveremos a clusterizar los datos de k-means, pero usando hierarchical clustering.

In [ ]:
from sklearn import preprocessing

#Cargando y viendo los datos
#En el caso de google colab tiene que subir el archivo a la nube primero
bank = pd.read_csv('./data/bank.csv',sep=";")

#Extrayendo las variables nominales
bank_cust = bank[['job', 'marital', 'education', 'default', 'housing', 'loan','contact','month','poutcome']].copy()

#Agregando edad como variable categorica
bank_cust['edad'] = pd.cut(bank['age'], [0, 20, 30, 40, 50, 60, 70, 80, 90, 100], 
                                 labels=['0-20', '20-30', '30-40', '40-50','50-60','60-70','70-80', '80-90','90-100'])

bank_cust_orig=bank_cust.copy() #Copia de los datos de origen

le = preprocessing.LabelEncoder()
bank_cust = bank_cust.apply(le.fit_transform)

# Tomamos una muestra de 1000 datos (por temas de performance)
muestraDatos=bank_cust.sample(1000)
muestraDatos

In [ ]:
#Aplicamos la función de distancia de scipy correspondiente a la distancia de Hamming 
#("porcentaje" de valores en los que vectores difieren) 
from scipy.spatial.distance import pdist
distCategorica = pdist(muestraDatos, 'hamming')

#Entrenando el modelo
modelo = shc.linkage(distCategorica, method='ward')
plt.figure(figsize=(10, 7)) #Seteando el tamaño de la figura
objeto = shc.dendrogram(modelo) #Generando el dendrograma

In [ ]:
#Seleccionando clusters
clusters=shc.fcluster(modelo,t=3.0,criterion="distance")

#Veamos los centroides 
out = pd.DataFrame()
for col in range(len(muestraDatos.columns)):
    modeVector=muestraDatos.groupby(clusters).agg(pd.Series.mode)
    le.fit(bank_cust_orig.iloc[:,col])
    out[muestraDatos.columns[col]] = le.inverse_transform(modeVector.iloc[:,col])
out['count'] = np.unique(clusters, return_counts=True)[1]

In [ ]:
out